# Bilan Hydrique

Sources :

- Coefficients culturaux :
  - [ARDEPI](https://www.ardepi.fr/nos-services/vous-etes-irrigant/estimer-ses-besoins-en-eau/maraichage/)
  - [Chambre d’agriculture Nouvelle-Aquitaine](https://gironde.chambre-agriculture.fr/fileadmin/user_upload/Nouvelle-Aquitaine/100_Inst-Gironde/Documents/pdf_grandes-cultures_accompagnement-technique_mieux-irriguer/Messages_irrigation_2019/message_1/Tableau_Coefficients_Culturaux_Kc_.02.pdf)


In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [58]:
# Constantes
# Coefficients culturaux (KC) par culture et par stade
KC = {
    'Pomme de terre': {'Vegetation': 0.9, 'Maximale': 1.05}
}

# Réserve Utile (RU) par cm de terre fine (mm/cm de terre fine) en fonction de la texture du sol
RU_PAR_CM_DE_TF = {
    'Terres argileuses': 1.85,
    'Argiles sableuses': 1.7, 'Argiles sablo-limoneuses': 1.8, 'Argiles limono-sableuses': 1.8, 'Argiles limoneuses': 1.9,
    'Terres argilo-sableuses': 1.7, 'Terres argilo-limono-sableuses': 1.8, 'Terres argilo-limoneuses': 2.,
    'Terres sablo-argileuses': 1.4, 'Terres sablo-limono-argileuses': 1.5, 'Terres limono-sablo-argileuses': 1.65, 'Terres limono-argileuses': 2.00,
    'Terres sableuses': 0.7, 'Terres sablo-limoneuses': 1., 'Terres limono-sableuses': 1.55, 'Terres limoneuses': 1.8
}

# Profondeurs d'enracinement typiques
PROFONDEUR_ENRACINEMENT_TYPIQUE = {
    'Radis': 15.,
    'Salade': 15.,
    'Choux': 20.,
    'Epinard': 20.,
    'Oignon': 20.,
    'Aubergine': 30.,
    'Carotte': 30.,
    'Courge': 30.,
    'Courgette': 30.,
    'Poivron': 30.,
    'Pomme de terre': 30.,
    'Tomate': 30.
}   

# Liste des variables météorologiques utilisées dans les calculs d'ETP et de bilan hydrique
LISTE_VARIABLES_METEO = ['ff', 't', 'u', 'ray_glo01', 'rr1']

In [3]:
# Données météorologiques
# Évapotranspiration Potentielle (ETP) sur la période (mm)
ETP = 28.

# Réserve Facilement Utilisable (RFU) initiale (mm)
RFU_INITIALE = 40.

# Pluviométrie (mm)
PLUVIOMETRIE = 5.

In [4]:
# RFU finale cible (mm)
RFU_FINALE_CIBLE = 40.

# Coefficient de conversion de la RU en RFU (entre 1/2 et 2/3)
RU_VERS_RFU = 2. / 3

# Fraction de la réserve utile du sol remplie d'eau (entre 0 pour une période sèche et 1 pour une période pluvieuse)
FRACTION_REMPLIE = 1.

# Fraction du sol occupé par des cailloux et graviers (entre 0 pour absence de cailloux et 1 pour totalité de cailloux)
FRACTION_CAILLOUX = 0.3

In [5]:
# Choix de la culture
culture = 'Pomme de terre'

# Choix du stade
stade = 'Vegetation'

# Choix de la texture
texture = 'Terres limoneuses'

In [6]:
# RU par cm de terre fine pour cette texture
ru_par_cm_de_tf_texture = RU_PAR_CM_DE_TF[texture]

# Calcul de la profondeur de terre fine
profondeur = PROFONDEUR_ENRACINEMENT_TYPIQUE[culture]
profondeur_tf = profondeur * (1. - FRACTION_CAILLOUX)

# Calcul de la RU (mm)
ru = ru_par_cm_de_tf_texture * profondeur_tf

# Calcul de la RFU maximale (mm)
rfu_max = ru * RU_VERS_RFU

# Calcul de la RFU disponible (mm)
rfu = rfu_max * FRACTION_REMPLIE

print(f"RFU pour une profondeur d'enracinement de {profondeur:.0f} cm : {rfu:.0f} mm")

RFU pour une profondeur d'enracinement de 30 cm : 25 mm


In [7]:
# KC de la culture pour ce stade
kc_culture = KC[culture][stade]

# Calcul de l'évalotranspiration maximale (mm)
etm_culture = kc_culture * ETP

print(f'Évapotranspiration maximale pour la {culture} au stade {stade}: {etm_culture:.0f} (mm)')

Évapotranspiration maximale pour la Pomme de terre au stade Vegetation: 25 (mm)


In [8]:
# Calcul de la RFU finale
besoin_irrigation = RFU_FINALE_CIBLE + etm_culture - (rfu + PLUVIOMETRIE)

print(f'Besoin en irrigation pour la {culture} au stade {stade}: {besoin_irrigation:.0f} (mm)')

Besoin en irrigation pour la Pomme de terre au stade Vegetation: 35 (mm)


In [9]:
# Définition de la station de référence
REF_STATION_NAME = 'La Petite Claye'
REF_STATION_LATLON = [48.541356, -1.615400]

In [10]:
from io import StringIO
import json
import requests

# Definitions pour accéder à l'API Météo-France via un token OAuth2
# unique application id : you can find this in the curl's command to generate jwt token 
APPLICATION_ID = 'ZlFGb1VCNzdlQ3c5QmhSMU1IbE8xQTluOE0wYTpUS3l1YkcweGJmSTJrQlJVaGNiSkNHTXczdHNh'

# url to obtain acces token
TOKEN_URL = "https://portail-api.meteofrance.fr/token"

class Client(object):
    def __init__(self):
        self.session = requests.Session()
        
    def request(self, method, url, **kwargs):
        # First request will always need to obtain a token first
        if 'Authorization' not in self.session.headers:
            self.obtain_token()
            
        # Optimistically attempt to dispatch reqest
        response = self.session.request(method, url, **kwargs)

        if self.token_has_expired(response):
            # We got an 'Access token expired' response => refresh token
            self.obtain_token()

            # Re-dispatch the request that previously failed
            response = self.session.request(method, url, **kwargs)

        return response

    def token_has_expired(self, response):
        status = response.status_code
        content_type = response.headers['Content-Type']
        repJson = response.text

        if status == 401 and 'application/json' in content_type:
            repJson = response.text
            
            if 'Invalid JWT token' in repJson['description']:
                return True

        return False

    def obtain_token(self):
        # Obtain new token
        data = {'grant_type': 'client_credentials'}
        headers = {'Authorization': 'Basic ' + APPLICATION_ID}
        access_token_response = requests.post(
            TOKEN_URL, data=data, verify=False, allow_redirects=False, headers=headers)
        token = access_token_response.json()['access_token']

        # Update session with fresh token
        self.session.headers.update({'Authorization': 'Bearer %s' % token})

def response_text_to_frame(response, **read_csv_kwargs):
    return pd.read_csv(StringIO(response.text), sep=';', **read_csv_kwargs)

In [11]:
# Initialisation d'un client pour accéder à l'API Météo-France
client = Client()

# Issue a series of API requests an example. For use this test, you must first subscribe to the arome api with your application
client.session.headers.update({'Accept': '*/*'})

In [12]:
# Host
METEOFRANCE_HOST = 'https://public-api.meteofrance.fr'
METEOFRANCE_DOMAIN = 'public'
METEOFRANCE_VERSION = 'v1'
METEOFRANCE_FREQUENCE = 'horaire'
METEOFRANCE_DIR_LISTE_STATIONS = 'liste-stations'
METEOFRANCE_API_STATION = ''
METEOFRANCE_GRANULARITE_STATION = 'station'
METEOFRANCE_API_PAQUET = 'DPPaquetObs'
METEOFRANCE_GRANULARITE_PAQUET = 'paquet'
METEOFRANCE_FMT = 'csv'

# URL de la liste des stations
URL_LISTE_STATIONS_STATION = (f'{METEOFRANCE_HOST}/{METEOFRANCE_DOMAIN}/{METEOFRANCE_API_STATION}/'
                              f'{METEOFRANCE_VERSION}/{METEOFRANCE_DIR_LISTE_STATIONS}')
URL_LISTE_STATIONS_PAQUET = (f'{METEOFRANCE_HOST}/{METEOFRANCE_DOMAIN}/{METEOFRANCE_API_PAQUET}/'
                             f'{METEOFRANCE_VERSION}/{METEOFRANCE_DIR_LISTE_STATIONS}')

# URL de base des données horaires
URL_DONNEE_STATION = (f'{METEOFRANCE_HOST}/{METEOFRANCE_DOMAIN}/{METEOFRANCE_API_STATION}/'
                      f'{METEOFRANCE_VERSION}/{METEOFRANCE_GRANULARITE_STATION}/{METEOFRANCE_FREQUENCE}')
URL_DONNEE_PAQUET = (f'{METEOFRANCE_HOST}/{METEOFRANCE_DOMAIN}/{METEOFRANCE_API_PAQUET}/'
                     f'{METEOFRANCE_VERSION}/{METEOFRANCE_GRANULARITE_PAQUET}/{METEOFRANCE_FREQUENCE}')

# Étiquettes de la latitude et de la longitude
LATLON_LABELS = ['Latitude', 'Longitude']

In [13]:
# Choix de l'api
url_liste_stations = URL_LISTE_STATIONS_PAQUET
url_donnee = URL_DONNEE_PAQUET

In [14]:
# Demande de la liste des stations
response_liste_stations = client.request('GET', url_liste_stations, verify=False)

/home/atantet/.conda/envs/agri_notebooks/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'portail-api.meteofrance.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/atantet/.conda/envs/agri_notebooks/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'public-api.meteofrance.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [15]:
# Conversion du texte répondu en DataFrame
df_liste_stations = response_text_to_frame(response_liste_stations, index_col='Id_station')

In [16]:
# Convert degrees to radians
latlon_rad_labels = []
for latlon_label, latlon_series in df_liste_stations[LATLON_LABELS].items():
    latlon_rad_label = f'{latlon_label}_rad'
    df_liste_stations[latlon_rad_label] = np.deg2rad(latlon_series)
    latlon_rad_labels.append(latlon_rad_label)
ref_station_latlon_rad = np.deg2rad(REF_STATION_LATLON)

In [17]:
from sklearn.neighbors import BallTree

# Calcul de l'arbre des plus proches voisins pour la liste des stations
X = df_liste_stations[latlon_rad_labels].values
tree = BallTree(X, metric='haversine')

In [18]:
# Rayon de la terre (km)
RAYON_TERRE_KM = 6371.

# # Identification d'un certain nombre de plus proches voisins
# NN_NUMBER = 20
# dist_rad_arr, ind_arr = tree.query([ref_station_latlon_rad], k=NN_NUMBER)

# Alternative : identification des plus proches voisins dans un certain rayon
NN_RAYON_KM = 35.
nn_rayon_rad = NN_RAYON_KM / RAYON_TERRE_KM
ind_arr, dist_rad_arr = tree.query_radius(
    [ref_station_latlon_rad], nn_rayon_rad,
    count_only=False, return_distance=True, sort_results=True)

dist_rad, ind = dist_rad_arr[0], ind_arr[0]

# Conversion en km de la distance en rad
dist_km = np.round(dist_rad * RAYON_TERRE_KM).astype(int)

In [91]:
# Sélection des plus proches voisins
df_liste_stations_nn = df_liste_stations.iloc[ind]
df_liste_stations_nn.name = REF_STATION_NAME
s_dist_km = pd.Series(dist_km, index=df_liste_stations_nn.index, name=REF_STATION_NAME)

print(f'{REF_STATION_NAME} ({REF_STATION_LATLON[0]:.6f}, {REF_STATION_LATLON[1]:.6f}) est à :')
for k, (nn_id, nn_series) in enumerate(df_liste_stations_nn.transpose().items()):
    station_name = df_liste_stations_nn.loc[nn_id, 'Nom_usuel']
    print(f'- {s_dist_km.loc[nn_id]:02d} km du {k}NN {station_name} '
          f'({nn_series[LATLON_LABELS[0]]:.6f}, {nn_series[LATLON_LABELS[1]]:.6f})')

La Petite Claye (48.541356, -1.615400) est à :
- 06 km du 0NN BROUALAN (48.485667, -1.640833)
- 09 km du 1NN PONTORSON (48.585667, -1.505167)
- 17 km du 2NN PLERGUER (48.524833, -1.843667)
- 24 km du 3NN FEINS  SA (48.326833, -1.596833)
- 27 km du 4NN PLESDER (48.406833, -1.924833)
- 29 km du 5NN MEZIERES-SUR-C. (48.308833, -1.439000)
- 31 km du 6NN ST OVIN (48.682500, -1.248667)
- 33 km du 7NN GRANVILLE (48.834500, -1.613667)
- 34 km du 8NN DINARD (48.584833, -2.076333)


In [20]:
def demande_donnee_station_date(id_station, date, verify=False):
    '''Demande des données horaires pour une station et une date données.'''
    # Paramètres définissant la station, la date et le format des données
    params = {'id_station': id_station, 'date': date, 'format': METEOFRANCE_FMT}

    # Demande de la liste des stations
    response = client.request(
        'GET', URL_DONNEE_STATION, verify=verify, params=params)

    return response

def compiler_donnee_des_stations_date(df_liste_stations, date):
    df = pd.DataFrame(dtype=float)
    for id_station in df_liste_stations.index:
        # Requête pour la station
        response = demande_donnee_station_date(id_station, date)

        # DataFrame de la station
        s_station = response_text_to_frame(response).iloc[0]
        df_station = s_station.to_frame(id_station).transpose()

        # Compilation
        df = pd.concat([df, df_station])
        
    return df

def demande_donnee_departement(id_departement, verify=False):
    '''Demande des données empaquetées pour un département.'''
    # Paramètres définissant le département et le format des données
    params = {'id-departement': id_departement, 'format': METEOFRANCE_FMT}

    # Demande de la liste des stations
    response = client.request(
        'GET', URL_DONNEE_PAQUET, verify=verify, params=params)

    return response

def compiler_donnee_des_departements(df_liste_stations):
    liste_id_departements = np.unique(
        [_ // 1000000 for _ in df_liste_stations.index])
    df_toutes = pd.DataFrame(dtype=float)
    for id_departement in liste_id_departements:
        # Requête pour la station
        response = demande_donnee_departement(id_departement)

        # DataFrame pour le département indexé par identifiant station et par date
        df_departement = response_text_to_frame(response).set_index(
            ['geo_id_insee', 'validity_time'])
        
        # Compilation
        df_toutes = pd.concat([df_toutes, df_departement])

    # Sélection des stations de la liste
    df = df_toutes.loc[df_liste_stations.index]

    # Réindexer à partir des noms usuels
    id_stations_df = df.index.to_frame()['geo_id_insee']
    liste_noms_stations = [df_liste_stations.loc[_, 'Nom_usuel']
                           for _ in id_stations_df]
    df.insert(0, 'Nom_usuel', liste_noms_stations)
        
    return df

In [21]:
df_meteo = compiler_donnee_des_departements(df_liste_stations_nn)[LISTE_VARIABLES_METEO]

/home/atantet/.conda/envs/agri_notebooks/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'public-api.meteofrance.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/atantet/.conda/envs/agri_notebooks/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'public-api.meteofrance.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [94]:
def interpolation_inverse_distance_carre(df, s_dist_km):
    '''Interpolation des plus proches voisins pondérée par l'inverse de la distance au carré.'''
    # Calcul des poids à partir des distances
    poids = 1. / s_dist_km**2

    # Adaptation des dimensions des poids aux données météo
    df_piv = df.unstack()
    poids_piv = (df_piv + 1.e-6).mul(poids, axis='index') / (df_piv + 1.e-6)

    # Interpolation
    df_ref = ((df_piv * poids_piv).sum(0) / poids_piv.sum(0)).unstack().transpose()
    
    df_ref.name = s_dist_km.name

    return df_ref

In [111]:
df_meteo_ref = interpolation_inverse_distance_carre(df_meteo, s_dist_km)

In [112]:
# Chaleur latente de vaporisation de l’eau (J kg-1)
LAMBDA = 2.45e6

# Constante psychométrique (kPa K-1)
GAMMA = 0.000665

# Constante de Stefan (W m-2 K-4)
SIGMA = 5.67e8 

# Albedo
ALPHA = 0.2

# Émissivité
EPSILON = 0.95

# ETP journalière maximale (mm j-1)
ETP_JOUR_MAX = 9.

# Rayonnement atmosphérique incident (J m-2)
RAYONNEMENT_ATMOSPHERIQUE = 0.

def calcul_etp(df):
    # Variable météorologiques
    # Vitesse du vent à 10 m (m s-1)
    vitesse_vent_10m = df['ff']

    # Temperature de l'air à 2 m (K)
    temperature_air_2m = df['t']

    # Humidité relative de l'air à 2 m
    humidite_relative_air_2m = df['u'] / 100

    # Rayonnement global (J m-2)
    rayonnement_global = df['ray_glo01']

    # Calcul de la pente de la courbe de pression de vapeur à la température moyenne de l'air (kPa K-1)
    exposant = 17.27 * (temperature_air_2m - 273.15) / (temperature_air_2m - 35.85)
    delta = 2504 * np.exp(exposant) / (temperature_air_2m - 35.85)**2

    # Calcul de rho * Cp / ra
    rho_cp_sur_ra = LAMBDA * GAMMA * 0.26 * (1. + 0.4 * vitesse_vent_10m)

    # Calcul de la pression de vapeur saturante (kPa)
    es = 0.6108 * np.exp(exposant)

    # Calcul de la pression de vapeur effective (kPa)
    ee = humidite_relative_air_2m * es

    # Calcul du rayonnement net
    rayonnement_net = ((1 - ALPHA) * rayonnement_global +
                       EPSILON * RAYONNEMENT_ATMOSPHERIQUE -
                       EPSILON * SIGMA * temperature_air_2m**4)

    # Calcul de l'ETP (mm h-1)
    denominateur = LAMBDA * (delta + GAMMA)
    etp1 = np.maximum(0, delta * rayonnement_net / denominateur)
    etp2 = np.maximum(0, rho_cp_sur_ra * (es - ee) / denominateur)
    etp = etp1 + etp2

    df_etp = etp.to_frame('etp')

    df_etp.name = df.name

    return df_etp

In [113]:
etp = calcul_etp(df_meteo_ref)
col = etp.columns[0]
df_meteo_ref[col] = etp

In [116]:
# Calcul de l'ETP journalière (mm j-1)
etp_jour = np.minimum(ETP_JOUR_MAX, etp.sum(0))
etp_jour

etp    0.01318
dtype: float64